# Production model: constraint-style LP with Python/Gurobi
## [Jon Lee](https://sites.google.com/site/jonleewebpage/home), 10/20/2020

Notes:
* This example is meant to show how to:
   * do constraint-style LP's (as opposed to matrix style), though the model we are setting up is $\max\{c'x ~:~ Ax\leq b,~ x\geq 0\}$.
   * extract primal and dual solutions, primal and dual slacks, and sensitivity information are printed
   * pass constraint names to Gurobi and then retrieve constraints from Gurobi by these names
   
References:
* Jon Lee, "A First Course in Linear Optimization", Fourth Edition (Version 4.0), Reex Press, 2013-20.

MIT License

Copyright (c) 2020 Jon Lee

Permission is hereby granted, free of charge, to any person obtaining a copy 
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [1]:
%reset -f
import numpy as np
import gurobipy as gp
from gurobipy import GRB

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [2]:
# Some toy data
m=3
n=2
M=list(range(0,m))
N=list(range(0,n))
A = np.array([ [8, 5], [8, 6], [8, 7] ])
b = np.array([32, 33, 35])
c = np.array([3 ,2])

In [3]:
model = gp.Model()
x = model.addMVar(n)
revenueobjective = model.setObjective(sum(c[j]*x[j] for j in N), GRB.MAXIMIZE)
for i in M:    # naming the constraints r0,r1,r2,... (inside Gurobi)
    model.addConstr(sum(A[i,j]*x[j] for j in N) <= b[i], name='r'+str(i)) 

Academic license - for non-commercial use only - expires 2021-08-26
Using license file C:\Users\jonxlee\gurobi.lic
Discarded solution information


In [4]:
model.optimize()
if model.status != GRB.Status.OPTIMAL:
    print("***** Gurobi solve status:", model.status)
    print("***** This is a problem. Model does not have an optimal solution")
    raise StopExecution
print(" ")
print("primal var,      dual slack,      obj delta-lb,    obj delta-ub")
for j in N: print("x[",j,"]=",np.format_float_positional(np.ndarray.item(x[j].X),4,pad_right=4), 
                  "  t[",j,"]=", np.format_float_positional(np.ndarray.item(x[j].RC),4,pad_right=4),
                  "  L[",j,"]=", np.format_float_positional(np.ndarray.item(x[j].SAObjLow-c[j]),4,pad_right=4),
                  "  U[",j,"]=", np.format_float_positional(np.ndarray.item(x[j].SAObjUp-c[j]),4,pad_right=4))
print(" ")
print("dual vars,       primal slack,    rhs delta-lb,    rhs delta-ub")
for i in M: 
    constr=model.getConstrByName('r'+str(i))    # retriving from Gurobi the named constraints r0,r1,r2,...
    print("y[",i,"]=",np.format_float_positional(constr.Pi,4,pad_right=4), 
                  "  s[",i,"]=", np.format_float_positional(constr.Slack,4,pad_right=4),
                  "  L[",i,"]=", np.format_float_positional(constr.SARHSLow-b[i],4,pad_right=4),
                  "  U[",i,"]=", np.format_float_positional(constr.SARHSUp-b[i],4,pad_right=4))

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x3e75bd93
Coefficient statistics:
  Matrix range     [5e+00, 8e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 4e+01]
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+30   5.250000e+30   5.000000e+00      0s
       3    1.2125000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds
Optimal objective  1.212500000e+01
 
primal var,      dual slack,      obj delta-lb,    obj delta-ub
x[ 0 ]= 3.3750   t[ 0 ]= 0.       L[ 0 ]= -0.3333   U[ 0 ]= 0.2   
x[ 1 ]= 1.       t[ 1 ]= 0.       L[ 1 ]= -0.125    U[ 1 ]= 0.25  
 
dual vars,       primal slack,    rhs delta-lb,    rhs delta-ub
y[ 0 ]= 0.2500   s[ 0 ]= 0.